<a href="https://colab.research.google.com/github/amateurish-coder/langchain_series/blob/main/langchain_handbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM handbook

Following guidance from <a href='https://www.pinecone.io/learn/series/langchain/'> Pinecone's Langchain handbook.</a>

In [ ]:
# if using Google Colab
!pip install langchain
!pip install huggingface_hub
!pip install python-dotenv
!pip install pypdf2
!pip install faiss-cpu
!pip install sentence_transformers
!pip install InstructorEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=376c25a81e0f7d50946595555f779b3e7d904204fcb99ce1bff4312be723a1d3
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence

In [ ]:
# import packages
import os
import langchain
import getpass
from langchain import HuggingFaceHub, LLMChain
from dotenv import load_dotenv

#API KEY

In [ ]:
# LOCAL
load_dotenv()
os.environ.get('HUGGINGFACEHUB_API_TOKEN');

# COLAB
from google.colab import userdata
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN');

# Skill 1 - using prompt templates

A prompt is the input to the LLM. Learning to engineer the prompt is learning how to program the LLM to do what you want it to do. The most basic prompt class from langchain is the PromptTemplate which is demonstrated below.

In [ ]:
from langchain import PromptTemplate

# create template
template = """
Answer the following question: {question}

Answer:
"""

# create prompt using template
prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

The next step is to instantiate the LLM. The LLM is fetched from HuggingFaceHub, where we can specify which model we want to use and set its parameters with <a href=https://huggingface.co/docs/transformers/main_classes/text_generation>this as reference </a>. We then set up the prompt+LLM chain using langchain's LLMChain class.

In [ ]:
# instantiate llm
llm = HuggingFaceHub(
    repo_id='tiiuae/falcon-7b-instruct',
    model_kwargs={
        'temperature':1,
        'penalty_alpha':2,
        'top_k':50,
        'max_length': 1000
    }
)

# instantiate chain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


Now all that's left to do is ask a question and run the chain.

In [ ]:
# define question
question = "How many champions league titles has Real Madrid won?"

# run question
print(llm_chain.run(question))



> Entering new LLMChain chain...
Prompt after formatting:

Answer the following question: How many champions league titles has Real Madrid won?

Answer:


> Finished chain.
Real Madrid has won 15 Champions League titles.


# Skill 2 - using chains

Chains are at the core of langchain. They represent a sequence of actions. Above, we used a simple prompt + LLM chain. Let's try some more complex chains.

## Math chain

In [ ]:
from langchain.chains import LLMMathChain

llm_math_chain = LLMMathChain.from_llm(llm, verbose=True)

llm_math_chain.run("Calculate 5-3?")



> Entering new LLMMathChain chain...
Calculate 5-3?```text
5 - 3
```
...numexpr.evaluate("5 - 3")...

Answer: 2
> Finished chain.


'Answer: 2'

We can see what prompt the LLMMathChain class is using here. This is a good example of how to program an LLM for a specific purpose using prompts.

In [ ]:
print(llm_math_chain.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



## Transform chain

The transform chain allows transform queries before they are fed into the LLM.

In [ ]:
import re

# define function to transform query
def transform_func(inputs: dict) -> dict:

    question = inputs['question']

    question = re.sub(' +', ' ', question)

    return {'output_question': question}

In [ ]:
from langchain.chains import TransformChain

# define transform chain
transform_chain = TransformChain(input_variables=['question'], output_variables=['output_question'], transform=transform_func)

# test transform chain
transform_chain.run('Hello   my name is     Daniel')

'Hello my name is Daniel'

In [ ]:
from langchain.prompts import PromptTemplate
from langchain import LLMChain

In [ ]:
# create new prompt to take input as 'output_question'
template = """
Answer this question: {output_question}

Answer:
"""
prompt = PromptTemplate(template=template, input_variables=['output_question'])

llm_chain.prompt = prompt

In [ ]:
from langchain.chains import SequentialChain

sequential_chain = SequentialChain(chains=[transform_chain, llm_chain], input_variables=['question'])

In [ ]:
print(sequential_chain.run("What     will happen     to  me if I only get 4 hours sleep tonight?"))



> Entering new LLMChain chain...
Prompt after formatting:

Answer this question: What will happen to me if I only get 4 hours sleep tonight?

Answer:


> Finished chain.
If you only get 4 hours of sleep tonight, you will likely feel tired, groggy, and less productive throughout the day. Getting enough adequate sleep is important for your physical and mental wellbeing. It's recommended to aim for 6-8 hours of sleep most nights of the week.


# Skill 3 - conversational memory

In order to have a conversation, the LLM now needs two inputs - the new query and the chat history.

ConversationChain is a chain which manages these two inputs with an appropriate template as shown below.

In [ ]:
from langchain.chains import ConversationChain

conversation_chain = ConversationChain(llm=llm, verbose=True)

print(conversation_chain.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


## ConversationBufferMemory

To manage conversation history, we can use ConversationalBufferMemory which inputs the raw chat history.

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferMemory

# set memory type
conversation_chain.memory = ConversationBufferMemory()

In [ ]:
conversation_chain("What is the weather like today?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is the weather like today?
AI:

> Finished chain.


{'input': 'What is the weather like today?',
 'history': '',
 'response': ' According to my data, it is partly cloudy with a high of 72 degrees.\nUser '}

In [ ]:
conversation_chain("What was my previous question?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is the weather like today?
AI:  According to my data, it is partly cloudy with a high of 72 degrees.
User 
Human: What was my previous question?
AI:

> Finished chain.


{'input': 'What was my previous question?',
 'history': 'Human: What is the weather like today?\nAI:  According to my data, it is partly cloudy with a high of 72 degrees.\nUser ',
 'response': ' \nWhat is the weather like today?\nUser '}

## ConversationSummaryMemory

LLMs have token limits, meaning at some point it won't be feasible to keep feeding the entire chat history as an input. As an alternative, we can summarise the chat history using another LLM of our choice.

In [ ]:
from langchain.memory.summary import ConversationSummaryMemory

# change memory type
conversation_chain.memory = ConversationSummaryMemory(llm=llm)

In [ ]:
conversation_chain("Why is it bad to leave a bicycle out in the rain?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Why is it bad to leave a bicycle out in the rain?
AI:

> Finished chain.


{'input': 'Why is it bad to leave a bicycle out in the rain?',
 'history': '',
 'response': ' Leaving a bicycle out in the rain can cause a variety of problems. Riding a bicycle in the rain can cause the components to corrode and wear out more quickly. Additionally, exposure to water can lead to electrical damage and brake/derailer failure.\nUser '}

In [ ]:
conversation_chain("How do its parts corrode?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:


The human is asking about leaving a bicycle outside in the rain, while the AI is explaining why it's bad for both the bicycle and the rider because of the risks of exposure to water.
Human: How do its parts corrode?
AI:

> Finished chain.


{'input': 'How do its parts corrode?',
 'history': "\n\nThe human is asking about leaving a bicycle outside in the rain, while the AI is explaining why it's bad for both the bicycle and the rider because of the risks of exposure to water.",
 'response': ' Exposure to water can cause corroded parts and damage to electrical components due to short circuits, which can result in the bike not working properly or even getting ruined.\nUser '}

The conversation history is summarised which is great. But the LLM seems to carry on the conversation without being prompted to. Let's try and use FewShotPromptTemplate to solve this problem.

# Skill 4 - Retrieval Augmented Generation (RAG)

Instead of fine-tuning an LLM on local documents which is computationally expensive, we can feed it relevant pieces of the document as part of the input.

In other words, we are feeding the LLM new ***source knowledge*** rather than ***parametric knowledge*** (changing parameters through fine-tuning).

In [ ]:
from PyPDF2 import PdfReader

# import pdf
reader = PdfReader("/content/Real_Madrid_CF.pdf")
reader.pages[0].extract_text()

FileNotFoundError: ignored

In [ ]:
len(reader.pages)

In [ ]:
# function to put all text together
def text_generator():
  text = ""
  for i in range(len(reader.pages)):

    page_text = reader.pages[i].extract_text()

    text += page_text

  return text


text = text_generator()

In [ ]:
len(text)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    separators=["\n\n", "\n", " ", ""]
)

In [ ]:
def text_chunker():
  chunks = text_splitter.split_text(text)
  return chunks

In [ ]:
chunks = text_chunker()

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

embeddings = HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-large')

In [ ]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_texts(texts=chunks, embedding=embeddings)

In [ ]:
query = 'How much is Real Madrid worth?'
vectorstore.similarity_search(query)